# HashiCorp Terraform Demo for Flexible Deployment Options (FDO) - Deploy Terraform Enterprise (TFE) on Kubernetes (K8s)

This is still WIP.  Not fully working.

This demo shows how the new TFE FDO (introduced in v202309-1) can be used to deploy a TFE setup in External Services mode on Kubernetes.   This means that the TFE data is stored externally of the TFE nodes.  This includes:
- PostgreSQL database (for storing metadata)
- S3-compatible blob storage (for storing terraform state files, configurations)
- Redis (cache for running transactions)

For performance reasons, the demo configures a single TFE node.  However, note that it can easily scale to 2 or more nodes as needed by modifying the number of replicas in the helm chart config.

Ref:
- https://developer.hashicorp.com/terraform/enterprise/replicated/install/operation-modes#operational-modes
- https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/kubernetes/requirements
- https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/kubernetes/install
- https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/configuration
- https://github.com/hashicorp/terraform-enterprise-helm/blob/main/values.yaml


The diagram below is a visual representation of this configuration.
<br>
<br>

<img src="">

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault).  This will be used for PKI certificate generation.
- PostgreSQL installed on docker (to simulate an external PostgreSQL database for the TFE setup)
- MinIO installed on docker (to simulate the external S3-compatible blob storage)
- Minikube (to simulate a K8s cluster)

Note that we will be using the docker and minikube routing to the host machine for communication between the docker postgres/minio pods and the Kubernetes cluster.  The host machine will function as a network bridge for communication.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# Install minikube
brew install minikube

In [ ]:
# Install Kubectl CLI
brew install kubernetes-cli

In [ ]:
# Install Helm CLI.  This is used to install the VSO helm chart.
brew install helm

In [ ]:
# Install K9s.  This is a nice console GUI for K8s.  https://k9scli.io/
brew install K9s

In [ ]:
# Install latest MinIO client
brew install minio/stable/mc

In [ ]:
# Install AWS CLI
brew install awscli

In [ ]:
# Installing Docker Desktop will add this entry in your /etc/hosts file
#   127.0.0.1 kubernetes.docker.internal
# Verify that kubernetes.docker.internal is resolvable to 127.0.0.1 loopback address
# We will using this to connect to the TFE setup with TLS
ping -c 3 kubernetes.docker.internal

# Also verify that port 443 is not used on kubernetes.docker.internal
# You should get a "Connection refused" error.
curl -v telnet://kubernetes.docker.internal:443

# Setting Up the Vault, PostgreSQL servers and K8s cluster
## Setup Vault for PKI certificate services

In [ ]:
# For this demo, we will be simulating a Vault server that is hosted external from the K8s cluster.  i.e. in Docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)

# Refresh Vault docker image with latest version
#docker pull hashicorp/vault-enterprise

# Optional: To pull latest image
#docker pull hashicorp/vault-enterprise:latest

# Run Vault in docker in Dev mode with Enterprise license.
# We have set VAULT_LOG_LEVEL to trace for troubleshooting purposes.  This will allow you to view detailed information as you test.
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-e "VAULT_LOG_LEVEL=trace" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

In [ ]:
# Optional: You can enable file audit device for more information
docker exec -it vault-enterprise /bin/sh -c "mkdir /var/log/vault.d"
docker exec -it vault-enterprise /bin/sh -c "touch /var/log/vault.d/vault_audit.log"
docker exec -it vault-enterprise /bin/sh -c "chown -R vault:vault /var/log/vault.d"
vault audit enable file file_path=/var/log/vault.d/vault_audit.log

# You can run the following command in the container terminal to follow the logs
# tail -f /var/log/vault.d/vault_audit.log
# Or you can run it from outside on your host machine
# docker exec -it vault-enterprise /bin/sh -c "tail -f /var/log/vault.d/vault_audit.log"
# Use Ctrl + C to break

## Setup TFE External Services

TFE External Services covers:
- PostgreSQL
- Blob Storage
- Redis

Ref: https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/requirements/data-storage/postgres-requirements

In [ ]:
# Run a PostgreSQL database for TFE storage
# Make sure it is a supported version.
# Ref: https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/requirements/data-storage/postgres-requirements
export PGUSER=root
export PGPASSWORD=mypassword
export PGPORT=5432

# Optional: To pull latest image
#docker pull postgres:15

# Run latest 15.x version
docker run --name postgres \
     -p $PGPORT:$PGPORT \
     --rm \
     -e POSTGRES_USER=$PGUSER \
     -e POSTGRES_PASSWORD=$PGPASSWORD \
     -d postgres:15

In [ ]:
# Run a MinIO container for the S3 compatible blob storage
# Ref: https://min.io/docs/minio/container/index.html
export MINIO_ROOT_USER=root
export MINIO_ROOT_PASSWORD=mypassword
export MINIO_API_PORT=9080 # I switched from default 9000 to 9080 as there is a Mac app that is running on that port after my Mac update
export MINIO_CONSOLE_PORT=9090

#docker stop minio
#rm -r ${HOME}/minio
# Create a path for the blob storage external to the container
mkdir -p ${HOME}/minio/data

# Optional: To pull latest image
#docker pull quay.io/minio/minio

docker run \
    -p $MINIO_API_PORT:$MINIO_API_PORT \
    -p $MINIO_CONSOLE_PORT:$MINIO_CONSOLE_PORT \
    --name minio \
    --rm \
    -v ${HOME}/minio/data:/data \
    --user $(id -u):$(id -g) \
    -e "MINIO_ROOT_USER=$MINIO_ROOT_USER" \
    -e "MINIO_ROOT_PASSWORD=$MINIO_ROOT_PASSWORD" \
    -d quay.io/minio/minio server /data --address ":$MINIO_API_PORT" --console-address ":$MINIO_CONSOLE_PORT"
# You can configure MinIO to use TLS
#  --certs-dir value, -S value  path to certs directory (default: "${HOME}/.minio/certs")
# Ref: https://min.io/docs/minio/container/operations/network-encryption.html


In [ ]:
# Run a redis container for the TFE cache
# Make sure it is a supported version.
# Ref: https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/requirements/data-storage/operational-mode-requirements#active-active-mode
export REDIS_PASSWORD=mypassword
export REDIS_PORT=6379

# Optional: To pull latest image
#docker pull redis:7

# We will be using Redis v7
docker run \
    -p $REDIS_PORT:$REDIS_PORT \
    --name redis \
    --rm \
    -e "MINIO_ROOT_USER=$MINIO_ROOT_USER" \
    -e "MINIO_ROOT_PASSWORD=$MINIO_ROOT_PASSWORD" \
    -d redis:7 redis-server --requirepass $REDIS_PASSWORD


In [ ]:
# Start minikube with 1 node for the demo.  Should be around 1-2 minutes to start.
# You can increase more nodes as required (to show a HA setup) but it will increase the start time for minikube
# and also the loading time of the TFE containers in the later steps.
minikube start --nodes 1

In [ ]:
# Verify minikube, Redis, MinIO, PostgreSQL, and Vault containers are running
docker ps

In [ ]:
# Verify that the K8s setup is working
kubectl get pods -A
echo
kubectl get svc -A

In [ ]:
# Optional: Enable metrics on minikube
minikube addons enable metrics-server

In [ ]:
# Optional: Run the following commands to check on the metrics
# Note: metrics-server will take a few minutes to start and be functional.  
# Try again later if it says "error: Metrics API not available"
kubectl top pods
echo
kubectl top nodes

## Verify TFE External Services

Make sure that all external services are working correctly before setting up TFE.

In [ ]:
# Verify that PostgreSQL is running properly

# Setup psql alias to the container to make it easier to do CLI commands to the postgresql pod
alias psql="docker exec -it postgres psql"

# Check connection info
psql -c '\conninfo'

In [ ]:
# Verify that MinIO is running properly
# Setup your MinIO client to point to your new MinIO setup with an alias called "myminio"
export MINIO_ALIAS=myminio
export MINIO_API_ADDR=http://127.0.0.1:$MINIO_API_PORT

echo "Connecting to $MINIO_API_ADDR"
echo "MinIO User: $MINIO_ROOT_USER"
echo "MinIO Password: $MINIO_ROOT_PASSWORD"
mc alias set $MINIO_ALIAS $MINIO_API_ADDR $MINIO_ROOT_USER $MINIO_ROOT_PASSWORD
mc admin info $MINIO_ALIAS

# For minio health checks, you can use this endpoint to check for 200 code.
#curl -I http://127.0.0.1:$MINIO_API_PORT/minio/health/live

In [ ]:
# Optional: Verify Redis is running properly
# Authenticate, set and get a value.
curl telnet://127.0.0.1:$REDIS_PORT << EOF
AUTH $REDIS_PASSWORD
SET TEST1 VALUE1
GET TEST1
DEL TEST1
GET TEST1
QUIT
EOF

# Setup Terraform Enterprise

## Configure PostgreSQL database

This requires creating a new database for TFE to install in.

In [ ]:
export TFE_DB_NAME=tfedb
psql -c "CREATE DATABASE $TFE_DB_NAME;"

# Verify that the tfedb is created
psql -P pager=off -c "\l"

## Configure a S3 bucket for TFE Object Storage

In [ ]:
# Generate a bucket name
export TFE_BUCKET_NAME="tfe-$(date +%s)"
# Set the S3 bucket region to the Minio default.
export TFE_BUCKET_REGION="us-east-1"

In [ ]:
# Create a S3 bucket on MinIO for storing TFE objects.
# Region on MinIO defaults to 'us-east-1'

# Generate a unique bucket name using prefix and unix epoch time
# Follow bucket naming conventions - https://docs.aws.amazon.com/AmazonS3/latest/userguide/bucketnamingrules.html
echo "MinIO User: $MINIO_ROOT_USER"
echo "MinIO Password: $MINIO_ROOT_PASSWORD"
# Remove the bucket if it exists
#mc rb $MINIO_ALIAS/$TFE_BUCKET_NAME
echo "Creating S3 bucket on MinIO: $TFE_BUCKET_NAME"
mc mb --region $TFE_BUCKET_REGION $MINIO_ALIAS/$TFE_BUCKET_NAME

In [ ]:
# Setup a S3 policy for TFE to use
tee pol-tfe-svc.json <<EOF
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:ListBucket",
                "s3:GetObject",
                "s3:DeleteObject",
                "s3:GetBucketLocation"
            ],
            "Resource": [
                "arn:aws:s3:::$TFE_BUCKET_NAME",
                "arn:aws:s3:::$TFE_BUCKET_NAME/*"
            ]
        }
    ]
}
EOF
# Additional permissions are needed by TFE to be able decrypt the bucket if S3 encryption is turned on
# MinIO Key Encryption Service (KES) supports S3 encryption with different KMS including HashiCorp Vault.
# Ref: https://min.io/docs/minio/linux/administration/server-side-encryption/server-side-encryption-sse-kms.html
# Ref: https://min.io/docs/minio/linux/operations/server-side-encryption/configure-minio-kes-hashicorp.html#minio-sse-vault
#     "Effect": "Allow",
#     "Action": [
#         "kms:Decrypt",
#         "kms:Encrypt",
#         "kms:DescribeKey",
#         "kms:ReEncrypt*",
#         "kms:GenerateDataKey*"
#     ],
#     "Resource": [
#         "<KMS_KEY_ARN>"
#     ]

mc admin policy create $MINIO_ALIAS pol-tfe-svc pol-tfe-svc.json
rm pol-tfe-svc.json

In [ ]:
# Verify that the new pol-tfe-svc S3 policy is created
mc admin policy list $MINIO_ALIAS

In [ ]:
# Create a new IAM programmatic credentials in MinIO with the created policy
export AWS_ACCESS_KEY_ID=$(openssl rand -base64 20 | tr -dc 'a-zA-Z0-9') # Generate random string of length 20
export AWS_SECRET_ACCESS_KEY=$(openssl rand -base64 20 | tr -dc 'a-zA-Z0-9')
mc admin user add $MINIO_ALIAS $AWS_ACCESS_KEY_ID $AWS_SECRET_ACCESS_KEY
mc admin policy attach $MINIO_ALIAS pol-tfe-svc --user $AWS_ACCESS_KEY_ID

In [ ]:
# Optional.  You can log into minio to view the create bucket, user, and policy.
echo "MinIO User: $MINIO_ROOT_USER"
echo "MinIO Password: $MINIO_ROOT_PASSWORD"
open http://127.0.0.1:9090

In [ ]:
# Verify that you can connect to the S3 bucket using the AWS CLI
echo "AWS_ACCESS_KEY_ID: $AWS_ACCESS_KEY_ID"
echo "AWS_SECRET_ACCESS_KEY: $AWS_SECRET_ACCESS_KEY"
echo "S3 endpoint: $MINIO_API_ADDR"

# unset the AWS_SESSION_TOKEN if it is set, otherwise there will be an error
unset AWS_SESSION_TOKEN

# Verify that the new credentials can see the TFE S3 bucket
echo
echo "Listing buckets:"
aws --region $TFE_BUCKET_REGION --endpoint-url $MINIO_API_ADDR s3 ls
echo
echo "Writing test file"
# Write a test file
echo "hello world" | aws --region $TFE_BUCKET_REGION --endpoint-url $MINIO_API_ADDR s3 cp - s3://$TFE_BUCKET_NAME/hello.txt
# List S3 bucket contents.  You should see the test file listed.
echo
echo "Listing bucket $TFE_BUCKET_NAME contents:"
aws --region $TFE_BUCKET_REGION --endpoint-url $MINIO_API_ADDR s3 ls s3://$TFE_BUCKET_NAME
echo
echo "Removing test file"
# Remove test file
aws --region $TFE_BUCKET_REGION --endpoint-url $MINIO_API_ADDR s3 rm s3://$TFE_BUCKET_NAME/hello.txt

## Generate PKI certificates for your TFE cluster

We will be using Vault's PKI engine to generate private TLS certificates for use by this TFE cluster.

In [ ]:
# Enable PKI engine at the default mount path
vault secrets enable pki

In [ ]:
# configure max lease ttl on certificates - 8760h or 365 days
vault secrets tune -max-lease-ttl=8760h pki

In [ ]:
# Create the CA, the private key is kept inside Vault.
# Alternatively, generate an intermediate CA and sign with your root CA.
# CA cert is valid for 8760h or 365 days
# As Docker Desktop configures kubernetes.docker.internal to resolve to the host,
# we will be using this domain and address for the TLS certificates.
export DOMAIN_NAME=docker.internal

vault write pki/root/generate/internal \
    common_name=$DOMAIN_NAME \
    ttl=8760h

In [ ]:
# configure Vault with the URL's for CRL
vault write pki/config/urls \
    issuing_certificates="$VAULT_ADDR/v1/pki/ca,host.minikube.internal:$VAULT_PORT" \
    crl_distribution_points="$VAULT_ADDR/v1/pki/crl,host.minikube.internal:$VAULT_PORT"

In [ ]:
# Store the CA certificate to import into your keychain.  Using keychain app to import in.

# Create folder to store certs
mkdir certs

# Export CA certificate
curl http://127.0.0.1:8200/v1/pki/ca_chain > certs/ca.crt
open certs/ca.crt

In [ ]:
# Note: To avoid cert warnings, open keychain app and configure CA cert for docker.internal to Always Trust in the properties
# Enter your password or Touch ID when prompted.
open -a "keychain Access.app"

<img src="images/trust-ca-cert.png">

In [ ]:
# Create the certs for the TFE setup
# For now, using the entry "kubernetes.docker.internal" configured by Docker Desktop to avoid DNS setup
echo "Domain Name: $DOMAIN_NAME"

export TFE_CLUSTER_NAME=kubernetes.$DOMAIN_NAME

vault write pki/roles/tfe-web-role \
    allowed_domains=$DOMAIN_NAME \
    allow_subdomains=true \
    max_ttl=8760h

In [ ]:
# Generate the web cert and place it in the nginx folder
echo "TFE Cluster Name: $TFE_CLUSTER_NAME"

vault write -format=json pki/issue/tfe-web-role \
    common_name=$TFE_CLUSTER_NAME > cert.json

jq -r .data.certificate < cert.json > certs/cert.pem
jq -r .data.private_key < cert.json > certs/cert.key



## Setup TFE Cluster with Helm

As we have prepared our external storage services and certificates, we can now proceed to setup TFE on the K8s cluster.

Ref: https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/kubernetes/install

In [ ]:
# Optional: View list of TFE versions
# Update the location of your license file as required
export TFE_LICENSE=$(cat $HOME/tfe-fdo-enterprise/terraform.hclic)
curl -s https://images.releases.hashicorp.com/v2/hashicorp/terraform-enterprise/tags/list -u terraform:$TFE_LICENSE \
| jq -r '[.tags[] | select (. | startswith("v"))] | sort_by(".") | reverse'

# Set TFE_VERSION to the version you wish to use

In [ ]:
# Log into the TFE FDO repo with your license and pull the TFE docker image
# Update the location of your license file as required
export TFE_LICENSE=$(cat $HOME/tfe-fdo-enterprise/terraform.hclic)
export TFE_VERSION=v202401-2

# Verify that you are able to log in and pull the docker image from your machine
echo $TFE_LICENSE |  docker login --username terraform images.releases.hashicorp.com --password-stdin
docker pull images.releases.hashicorp.com/hashicorp/terraform-enterprise:$TFE_VERSION

In [ ]:
# Pre-load the TFE docker image into the minikube nodes. Takes around 3-4 minutes on 1 worker node and will take longer with more nodes.
# This is to avoid image pull issues from the minikube nodes. 
minikube image load images.releases.hashicorp.com/hashicorp/terraform-enterprise:$TFE_VERSION

In [ ]:
# Verify the TFE image is loaded in minikube.  Images are cached in ~/.minikube/cache/images
minikube image list | grep terraform

In [ ]:
# Create a K8s namespace for your TFE cluster
export TFE_NAMESPACE=tfe-ns

kubectl create ns $TFE_NAMESPACE

In [ ]:
# Create an image pull secret in <TFE_NAMESPACE> to fetch the terraform-enterprise container from the <DOCKER_REGISTRY_URL>.
# This URL can be images.releases.hashicorp.com, or your internal container registry.  e.g. airgap setups.
# This step is important otherwise you will get an ImagePullErr on your TFE pod when running the helm installation
# (even if you have used minikube image load)
kubectl create secret docker-registry terraform-enterprise --docker-server=images.releases.hashicorp.com \
    --docker-username=terraform --docker-password=$TFE_LICENSE  -n $TFE_NAMESPACE

In [ ]:
# Add the HashiCorp repo (Only required for the first time)
helm repo add hashicorp https://helm.releases.hashicorp.com

In [ ]:
# Optional.  Update the repo (Only required when new versions are released)
helm repo update

In [ ]:
# Ref: https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/kubernetes/requirements
# As we are using MinIO for the S3 compatible storage, TFE_OBJECT_STORAGE_S3_USE_INSTANCE_PROFILE will be set to false.
echo "TFE Cluster Name: $TFE_CLUSTER_NAME"
echo "Installing in K8s namespace: $TFE_NAMESPACE"
echo "TFE Cluster name: $TFE_CLUSTER_NAME"
echo "TFE Database name: $TFE_DB_NAME"
echo "PostgreSQL Username: $PGUSER"
echo "PostgreSQL Username: $PGPASSWORD"
echo "PostgreSQL Port Number: $PGPORT"
echo "Redis Port Number: $REDIS_PORT"
echo "Redis Password: $REDIS_PASSWORD"
echo "S3 endpoint: $MINIO_API_ADDR"
echo "S3 Bucket name: $TFE_BUCKET_NAME"
echo "AWS_ACCESS_KEY_ID: $AWS_ACCESS_KEY_ID"
echo "AWS_SECRET_ACCESS_KEY: $AWS_SECRET_ACCESS_KEY"

# remove existing temporary values.yaml if it exists
rm values.yaml
# Set TFE encryption password
export TFE_ENCRYPTION_PASSWORD=encryptme
# For full list of parameters, see this.
# Ref: https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/configuration
# See helm chart default values
# Ref: https://github.com/hashicorp/terraform-enterprise-helm/blob/main/values.yaml
tee values.yaml <<EOF
replicaCount: 1
service:
  type: NodePort
  port: 443
  targetPort: 8443
  nodePort: 32443
tls:
  certData: $(cat certs/cert.pem | base64)
  keyData: $(cat certs/cert.key | base64)
  caCertData: $(cat certs/ca.crt | base64)
image:
  repository: images.releases.hashicorp.com
  name: hashicorp/terraform-enterprise
  tag: $TFE_VERSION
env:
  variables:
    TFE_HOSTNAME: $TFE_CLUSTER_NAME
    TFE_IACT_SUBNETS: 0.0.0.0/0
    TFE_DATABASE_HOST: host.minikube.internal:$PGPORT
    TFE_DATABASE_NAME: $TFE_DB_NAME
    TFE_DATABASE_PARAMETERS: sslmode=disable
    TFE_DATABASE_USER: $PGUSER
    TFE_REDIS_HOST: host.minikube.internal:$REDIS_PORT
    TFE_REDIS_USE_TLS: false
    TFE_REDIS_USE_AUTH: true
    TFE_OBJECT_STORAGE_TYPE: s3
    TFE_OBJECT_STORAGE_S3_USE_INSTANCE_PROFILE: false
    TFE_OBJECT_STORAGE_S3_BUCKET: $TFE_BUCKET_NAME
    TFE_OBJECT_STORAGE_S3_ENDPOINT: http://host.minikube.internal:$MINIO_API_PORT
    TFE_OBJECT_STORAGE_S3_REGION: $TFE_BUCKET_REGION
    TFE_CAPACITY_CONCURRENCY: 10
  secrets:
    TFE_DATABASE_PASSWORD: $PGPASSWORD
    TFE_OBJECT_STORAGE_S3_ACCESS_KEY_ID: $AWS_ACCESS_KEY_ID
    TFE_OBJECT_STORAGE_S3_SECRET_ACCESS_KEY: $AWS_SECRET_ACCESS_KEY
    TFE_REDIS_PASSWORD: $REDIS_PASSWORD
    TFE_LICENSE: $TFE_LICENSE
    TFE_ENCRYPTION_PASSWORD: $TFE_ENCRYPTION_PASSWORD
EOF
#    TFE_HTTP_PORT: 80
#    TFE_HTTPS_PORT: 443

# Render the terraform-enterprise chart with your custom values file
#helm template terraform-enterprise hashicorp/terraform-enterprise --values values.yaml -n $TFE_NAMESPACE 
# Install TFE with helm using the configuration values
# helm delete terraform-enterprise -n $TFE_NAMESPACE
helm install terraform-enterprise hashicorp/terraform-enterprise --values values.yaml -n $TFE_NAMESPACE 

In [ ]:
# Verify that the TFE deployment is up and running.  There is also a NodePort configuration configured
kubectl get deployments -n $TFE_NAMESPACE
echo
kubectl get pods -n $TFE_NAMESPACE
echo
kubectl get services -n $TFE_NAMESPACE

In [ ]:
# For troubleshooting, if the TFE pod is running you can use K9s to open a shell into the TFE pod to check the individual logs 
# Ref: https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/troubleshooting
open /opt/homebrew/bin/k9s
# Type '0' to view all namespaces
# Select the TFE pod and type 's' to open a shell
# Run this command to check the status of all TFE services.  All services should be running except postgres and redis.
# supervisorctl status
# All the logs for each component is here.  Check each log for errors.
# cd /var/log/terraform-enterprise
# Grep for all errors in the current folder for the word 'error' (case insensitive option)
# grep -r -i . -e 'error'
# Check for listening ports
# ss -tuln

## Setup Ingress on Minikube with the NGINX Ingress Controller

Ref: https://kubernetes.io/docs/tasks/access-application-cluster/ingress-minikube/


In [ ]:
# Step 1 - Enable the Ingress Controller
minikube addons enable ingress

In [ ]:
# Verify that the ingress controller pods are running in the ingress-nginx namespace
kubectl get pods -n ingress-nginx

In [ ]:
# The TFE installation configures the TLS certificates in a K8s secret called "terraform-enterprise-certificates"
# View TLS certificate details in K8s secret
kubectl -n $TFE_NAMESPACE describe secret terraform-enterprise-certificates
echo
kubectl -n $TFE_NAMESPACE get secrets terraform-enterprise-certificates --output=json | jq -r '.data'

In [ ]:
# For the Ingress configuration, we will be reusing the TLS certificate configured by the TFE setup
# Otherwise if not specified, Kubernetes Ingress Controller generates a fake test certificate

# Create an Ingress for TFE
echo "TFE Cluster Name: $TFE_CLUSTER_NAME"

kubectl apply -n $TFE_NAMESPACE  -f - <<EOF
apiVersion: networking.k8s.io/v1 
kind: Ingress 
metadata: 
  annotations: 
    nginx.ingress.kubernetes.io/force-ssl-redirect: "true"
    nginx.ingress.kubernetes.io/backend-protocol: "HTTPS"
  name: terraform-enterprise
  namespace: $TFE_NAMESPACE
spec: 
  rules: 
  - host: $TFE_CLUSTER_NAME
    http: 
      paths: 
      - backend: 
          service:
            name: "terraform-enterprise"
            port: 
              number: 443
        path: "/"
        pathType: "Prefix" 
  tls: 
  - hosts: 
    - $TFE_CLUSTER_NAME
    secretName: terraform-enterprise-certificates
EOF

In [ ]:
# Verify the ingress is created
kubectl get ingress -n $TFE_NAMESPACE

In [ ]:
# Important Step for the host to be able to access the TFE console!
# Run this in an external terminal window to expose port 443 to the host machine.
# Use hotkey Ctrl + Shift + ` to open terminal in VS Code (MacOS)
# Copy and run this command in the terminal window.  Do not run it here!

# minikube tunnel

# Note: You will need to key in your password


## Get the Initial Admin Creation Token (IACT) and Create the First TFE Admin User

In [ ]:
# Get name of first terraform pod
export TFE_POD_NAME=$(kubectl get pod -o jsonpath="{.items[0].metadata.name}" -n $TFE_NAMESPACE)

echo "TFE First Pod Name: $TFE_POD_NAME"

# Initial Admin Creation Token
# https://developer.hashicorp.com/terraform/enterprise/flexible-deployments/install/initial-admin-user

export IACT_TOKEN=$(kubectl exec -it $TFE_POD_NAME -n $TFE_NAMESPACE -- retrieve-iact)
echo "IACT Token: $IACT_TOKEN"

open https://$TFE_CLUSTER_NAME/admin/account/new?token=$IACT_TOKEN

# Enter initial admin details and create your first TFE organization.


# Cleanup

In [ ]:
# Cleanup

# Delete TFE
helm delete terraform-enterprise -n $TFE_NAMESPACE

# stop vault
docker stop vault-enterprise

# stop postgres
docker stop postgres

# stop redis
docker stop redis

# stop minio and clean up data folder
docker stop minio
rm -r ${HOME}/minio/data

# Clear up generated certs
rm -r certs

# Remove temp files
rm cert.json
rm values.yaml

# stop minikube
minikube delete

In [ ]:
# Manual Cleanup step
open -a "keychain Access.app"
# Remove the docker.internal certificate

In [ ]:
# Optional: Remove docker image in minikube.  This is useful if you are running out of disk space and want to clear older versions.
# Note that you still need to clear the image from ~/.minikube/cache/images
minikube image rm images.releases.hashicorp.com/hashicorp/terraform-enterprise:$TFE_VERSION
open ~/.minikube/cache/images

Lastly use Ctrl-C in the terminal window to exit out of the minikube tunnel

# End of Demo

# Appendix - Other Useful Commands

In [ ]:
# Optional: Add metrics-server to be able to view CPU and memory usage
helm repo add metrics-server https://kubernetes-sigs.github.io/metrics-server/
helm repo update
helm upgrade --install --set args={--kubelet-insecure-tls} metrics-server metrics-server/metrics-server --namespace kube-system

In [ ]:
# For testing connectivity.  Bash shell to a temporary pod and do curl commands
kubectl run my-shell --rm -i --tty --image ubuntu -- bash
apt update
apt install curl
curl http://host.minikube.internal:8200 

## Docker Commands

In [ ]:
# Docker network commands
# Internal host name - host.docker.internal
docker network ls
docker network connect <network> <container>
docker network create --driver bridge vso-connect

# Get IP address of Vault pod
docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' vault-enterprise


In [ ]:
# Reclaim space used by docker if you get a "no space left on device" error
docker system prune --all --force --volumes

In [ ]:
# Optional for troubleshooting common issues

# docker ps -a
# docker exec -it tfe-worker sh

# Image pull issues
# SSH into minikube.  Verify that you can pull the docker image.
echo "Run this command to log into an SSH shell on minikube"
echo
echo "minikube ssh -n minikube"
echo
echo "Run these commands in the minikube SSH shell to verify that you can login to the private repo and pull the image properly"
echo
echo "docker login --username terraform images.releases.hashicorp.com"
echo "docker pull images.releases.hashicorp.com/hashicorp/terraform-enterprise:$TFE_VERSION"
echo
echo "Enter this password when prompted:"
echo
echo $TFE_LICENSE

## K8s Commands

In [ ]:
# Get pods example
kubectl get pods -n $TFE_NAMESPACE

kubectl get pods -o wide -n $TFE_NAMESPACE  

In [ ]:
# View all K8s pod details - wide format
kubectl get pod -o wide -A

# Getting K8s secrets sample
kubectl get secrets -n $KUBENAMESPACE  --output=json | jq -r '.items[].metadata | select(.name|startswith("vault-token-")).name'

In [ ]:
# Delete deployment and services example
kubectl delete deployment -n $KUBENAMESPACE vso-db-demo
kubectl delete services -n $TFE_NAMESPACE terraform-enterprise

In [ ]:
# Clean up namespaces if they are not terminating gracefully
kubectl proxy &
kubectl get namespace tfe-ns-agents -o json |jq '.spec = {"finalizers":[]}' >temp.json
curl -k -H "Content-Type: application/json" -X PUT --data-binary @temp.json 127.0.0.1:8001/api/v1/namespaces/tfe-ns-agents/finalize

In [ ]:
# Display nodeports for terraform-enterprise
kubectl get service terraform-enterprise --output='jsonpath="{.spec.ports[0].nodePort}"' -n $TFE_NAMESPACE

In [ ]:
# Exposing Node Ports
kubectl expose deployment terraform-enterprise --type=NodePort --port=8080 -n $TFE_NAMESPACE
kubectl expose deployment terraform-enterprise --type=NodePort --port=8443 -n $TFE_NAMESPACE


In [ ]:
# Creating cert secret from files
kubectl -n $TFE_NAMESPACE create secret generic terraform-enterprise-certificates --from-file=./certs/

In [ ]:
# Deleting ingress
kubectl delete ingress -n $TFE_NAMESPACE terraform-enterprise

## minikube Commands

In [ ]:
# View add-ons
minikube addons list

In [ ]:
# Launch K8s dashboard
minikube dashboard

In [ ]:
# Follow minikube logs
new minikube logs -f

## kind Commands

In [ ]:
# Install kind
brew install kind

In [ ]:
# Start a kind cluster with 3 node (3 nodes for the TFE cluster)
# Ref: https://kind.sigs.k8s.io/docs/user/ingress/
export TFE_K8S_CLUSTER_NAME=tfe

kind create cluster --name $TFE_K8S_CLUSTER_NAME --image kindest/node:v1.28.0 --config - <<EOF
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
- role: control-plane
  kubeadmConfigPatches:
  - |
    kind: InitConfiguration
    nodeRegistration:
      kubeletExtraArgs:
        node-labels: "ingress-ready=true"
  extraPortMappings:
  - containerPort: 443
    hostPort: 443
    listenAddress: "0.0.0.0" # Optional, defaults to "0.0.0.0"
    protocol: tcp # Optional, defaults to tcp
- role: worker
- role: worker
- role: worker
EOF

In [ ]:
# Setup Ingress nginx for kind
kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/main/deploy/static/provider/kind/deploy.yaml

# Wait until ingress is ready to process requests
kubectl wait --namespace ingress-nginx \
  --for=condition=ready pod \
  --selector=app.kubernetes.io/component=controller \
  --timeout=90s

In [ ]:
# Load the TFE docker image into the kind nodes
kind load docker-image images.releases.hashicorp.com/hashicorp/terraform-enterprise:$TFE_VERSION -n $TFE_K8S_CLUSTER_NAME

In [ ]:
# Delete kind cluster
kind delete cluster --name $TFE_K8S_CLUSTER_NAME